## Práctica 5: Detección de características
Integrantes: Alejandro Bolaños Gracía y David García Díaz

### Ejercicio 1:
- a) A través de la interfaz modificar los parámetros del detector de características SIFT.
- b) Seleccionar un área de interés en una imagen de elección una imagen de naturaleza médica y
una imagen telemétrica.
- c) Buscar esa área de interés (recuádrela en rojo) dentro de diferentes versiones de la imagen de
partida (con cambios de traslación, escala y rotación) [NOTA: Estos cambios se pueden acometer
con un editor de imágenes o con el trabajo hecho en prácticas previas]. Altere mediante la interfaz
las configuraciones de parámetros para mejorar la detección.
- d) Pruebe a hacer lo mismo que en el apartado c) con diferentes grados de deformación de la
imagen. [NOTA: Estos cambios se pueden acometer con un editor de imágenes o con el trabajo
hecho en prácticas previas].


Para ejecutar el código se deben seguir estos pasos:
1. Ejecutas el bloque de código, a partir del cual se abrirá una interfaz gráfica.

2. Primero se debe abrir la imagen en cuestión de la cual vamos utilizar para el SIFT.

3. Puedes seleccionar la región de intéres con un rectángulo haciendo uso del ratón, o puedes también aplicar transformaciones a la imagen desde la interfaz gráfica, da igual el orden.

4. Una vez se hayan realizado las transformaciones y se haya seleccionado la región de interés, se puede presionar el botón para encontrar las características, también antes de ello puedes modificar los parámetros del SIFT en los trackbars de la interfaz gráfica.

5. Se mostrarán entonces mediante líneas uniendo ambas imágenes, las coincidencias entre la región rectangular y la imagen tranformada.

En caso de que se quiera realizar para una nueva imagen, se debe cerrar las imágenes mostradas, mediante la tecla $esc$, y acto seguido puede volver a abrir una imagen con el botón correspondiente de la interfaz gráfica y repetir el proceso mencionado anteriormente.

In [22]:
import cv2 as cv
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image
import matplotlib.pyplot as plt


# Variables globales para control de la imagen y transformaciones
drawing = False  
ix, iy = -1, -1  
fx, fy = -1, -1  
img = None  
cropped_image = None  
transformed_image = None  
w, h = 0, 0  

# Variables para transformaciones
tx, ty = 0, 0  
angle = 0  
scale_x = 1.0  
scale_y = 1.0  



# Función para actualizar todas las transformaciones de una vez y ajustar la ventana de visualización
def update_transformation():
    global img, transformed_image, tx, ty, angle, scale_x, scale_y, w, h

    if img is None:
        return

    # Escalar la imagen
    scaled_img = cv.resize(img, (0, 0), fx=scale_x, fy=scale_y)
    (h_scaled, w_scaled) = scaled_img.shape[:2]  
    center = (w_scaled // 2, h_scaled // 2)  

    # Crear la matriz de transformación para rotación y traslación
    rotation_matrix = cv.getRotationMatrix2D(center, angle, 1.0)
    rotation_matrix[0, 2] += tx  
    rotation_matrix[1, 2] += ty  

    # Aplicar rotación y traslación a la imagen escalada
    transformed_image = cv.warpAffine(scaled_img, rotation_matrix, (w_scaled, h_scaled), flags=cv.INTER_LINEAR, borderMode=cv.BORDER_CONSTANT, borderValue=(0, 0, 0))

    # Ajustar el tamaño de la ventana de visualización y mostrar la imagen transformada
    cv.namedWindow('Imagen Transformada', cv.WINDOW_NORMAL)
    cv.resizeWindow('Imagen Transformada', w_scaled, h_scaled)
    cv.imshow('Imagen Transformada', transformed_image)



# Función de callback para el ratón que permite seleccionar la región en la imagen
def draw_figures(event, x, y, flags, param):
    global ix, iy, fx, fy, drawing, img

    if event == cv.EVENT_LBUTTONDOWN:  
        drawing = True
        ix, iy = x, y 

    elif event == cv.EVENT_MOUSEMOVE:  
        if drawing:
            img_copy = img.copy()
            cv.rectangle(img_copy, (ix, iy), (x, y), (0, 0, 255), 1)
            cv.imshow('Imagen Original', img_copy)

    elif event == cv.EVENT_LBUTTONUP:  # 
        drawing = False
        fx, fy = x, y  # Coordenadas finales de la selección
        crop_image()  # Recortar la imagen en la región seleccionada



# Función para recortar la región seleccionada de la imagen
def crop_image():
    global img, ix, iy, fx, fy, cropped_image
    x1, y1 = min(ix, fx), min(iy, fy)
    x2, y2 = max(ix, fx), max(iy, fy)
    cropped_image = img[y1:y2, x1:x2]  # Recortar la imagen en la región seleccionada



# Función para emparejar características SIFT entre la región seleccionada y la imagen transformada
def sift_matches():
    global cropped_image, transformed_image
    if cropped_image is None or transformed_image is None:
        messagebox.showinfo("Error", "Primero selecciona una región y aplica una transformación.")
        return

    # Convertir las imágenes a escala de grises
    img1_gray = cv.cvtColor(cropped_image, cv.COLOR_BGR2GRAY)
    img2_gray = cv.cvtColor(transformed_image, cv.COLOR_BGR2GRAY)

    # Obtener los valores de los parámetros de SIFT desde los sliders
    sigma_value = max(0.1, float(sigma_scale.get()))
    sift = cv.SIFT_create(
        nfeatures=int(nfeatures_scale.get()),
        nOctaveLayers=int(nOctaveLayers_scale.get()),
        contrastThreshold=float(contrastThreshold_scale.get()),
        edgeThreshold=int(edgeThreshold_scale.get()),
        sigma=sigma_value
    )

    # Detectar y computar las características SIFT en ambas imágenes
    keypoints1, descriptors1 = sift.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = sift.detectAndCompute(img2_gray, None)

    # Emparejamiento de características usando BFMatcher
    bf = cv.BFMatcher(cv.NORM_L2, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)

    # Verificar si existen matches
    if len(matches) == 0:
        messagebox.showinfo("Error", "No se encontraron matches.")
        return

    # Ordenar y dibujar los mejores matches
    matches = sorted(matches, key=lambda x: x.distance)
    print(f"Total matches: {len(matches)}")
    img_matches = cv.drawMatches(cropped_image, keypoints1, transformed_image, keypoints2, matches[:50], None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    # Mostrar los matches en una ventana de OpenCV y en Matplotlib
    cv.imshow('Matches (SIFT)', img_matches)
    cv.waitKey(0)
    cv.destroyAllWindows()
    plt.imshow(cv.cvtColor(img_matches, cv.COLOR_BGR2RGB))
    plt.title('Matches (SIFT)')
    plt.show()



# Función para abrir y preprocesar una imagen desde un archivo
def open_file():
    global img, w, h
    file_path = filedialog.askopenfilename(filetypes=[("PNG Files", "*.png"), ("JPG Files", "*.jpg")])
    if file_path:
        img = Image.open(file_path).convert("RGB")  
        img = np.array(img)  
        img = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
        
        # Aplicar CLAHE (mejora del contraste) y suavizado con filtro Gaussiano
        clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        img_clahe = clahe.apply(img)
        img_smooth = cv.GaussianBlur(img_clahe, (5, 5), 1)
        img = cv.cvtColor(img_smooth, cv.COLOR_GRAY2BGR)
        
        h, w = img.shape[:2]
        cv.imshow('Imagen Original', img)
        cv.setMouseCallback('Imagen Original', draw_figures)



# Funciones de callback de sliders para actualizar las transformaciones
def on_trackbar_rotation(val):
    global angle
    angle = float(val)
    update_transformation()

def on_trackbar_translation_x(val):
    global tx
    tx = float(val) - 100
    update_transformation()

def on_trackbar_translation_y(val):
    global ty
    ty = float(val) - 100
    update_transformation()

def on_trackbar_resize_x(val):
    global scale_x
    scale_x = float(val) / 100
    update_transformation()

def on_trackbar_resize_y(val):
    global scale_y
    scale_y = float(val) / 100
    update_transformation()



# Configuración de la interfaz de Tkinter
window = tk.Tk()
window.title("Transformaciones y SIFT")
window.geometry("720x700")


# Trackbars para transformaciones
transform_label = tk.Label(window, text="Transformaciones", font=("Arial", 12, "bold"))
transform_label.place(x=20, y=20)

rotation_scale = tk.Scale(window, from_=0, to=360, orient=tk.HORIZONTAL, label="Rotación (grados)", command=on_trackbar_rotation)
rotation_scale.place(x=200, y=50, width=300)

translation_x_scale = tk.Scale(window, from_=0, to=200, orient=tk.HORIZONTAL, label="Traslación X", command=on_trackbar_translation_x)
translation_x_scale.place(x=400, y=120, width=300)

translation_y_scale = tk.Scale(window, from_=0, to=200, orient=tk.HORIZONTAL, label="Traslación Y", command=on_trackbar_translation_y)
translation_y_scale.place(x=400, y=190, width=300)

resize_x_scale = tk.Scale(window, from_=50, to=150, orient=tk.HORIZONTAL, label="Escala X (%)", command=on_trackbar_resize_x)
resize_x_scale.set(100)
resize_x_scale.place(x=20, y=120, width=300)

resize_y_scale = tk.Scale(window, from_=50, to=150, orient=tk.HORIZONTAL, label="Escala Y (%)", command=on_trackbar_resize_y)
resize_y_scale.set(100)
resize_y_scale.place(x=20, y=190, width=300)


# Trackbars para los parámetros de SIFT
sift_label = tk.Label(window, text="Parámetros de SIFT", font=("Arial", 12, "bold"))
sift_label.place(x=20, y=320)

nfeatures_scale = tk.Scale(window, from_=0, to=5000, orient=tk.HORIZONTAL, label="Número de características")
nfeatures_scale.set(0)
nfeatures_scale.place(x=20, y=360, width=300)

nOctaveLayers_scale = tk.Scale(window, from_=0, to=10, orient=tk.HORIZONTAL, label="Número de capas por octava")
nOctaveLayers_scale.set(3)
nOctaveLayers_scale.place(x=400, y=360, width=300)

contrastThreshold_scale = tk.Scale(window, from_=0, to=1, resolution=0.01, orient=tk.HORIZONTAL, label="Umbral de contraste")
contrastThreshold_scale.set(0.04)
contrastThreshold_scale.place(x=20, y=430, width=300)

edgeThreshold_scale = tk.Scale(window, from_=0, to=10, orient=tk.HORIZONTAL, label="Umbral de borde")
edgeThreshold_scale.set(5)
edgeThreshold_scale.place(x=400, y=430, width=300)

sigma_scale = tk.Scale(window, from_=0, to=10, resolution=0.1, orient=tk.HORIZONTAL, label="Sigma")
sigma_scale.set(1.6)
sigma_scale.place(x=200, y=500, width=300)


# Botones para abrir la imagen y realizar matching
open_button = tk.Button(window, text="Abrir imagen", command=open_file)
open_button.place(x=200, y=600, width=120, height=30)

sift_button = tk.Button(window, text="Encontrar matches (SIFT)", command=sift_matches)
sift_button.place(x=350, y=600, width=150, height=30)


# Ejecuta la ventana de Tkinter
window.mainloop()
